In [ ]:
path_to_module = '/content/drive/MyDrive/TESS-ZTF-Transient-Classification/'

import sys
import pandas as pd
import keras
import pickle
import numpy as np
from os import walk
import os
import matplotlib.pyplot as plt
import pickle as pkl
import math
import random
import keras.backend as K
import tensorflow as tf

from sys import path
from google.colab import drive, files
from collections import Counter
from warnings import filterwarnings
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras import layers, Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import (GRU, Dense, Lambda, Masking, RepeatVector, TimeDistributed, concatenate)
from keras.optimizers import adam_v2
from sklearn.manifold import TSNE
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()
filterwarnings("ignore")
path.append(path_to_module)
drive.mount('/content/drive/')

In [ ]:
# define constants
R_BAND = 6.5
G_BAND = 4.8
TESS_BAND = 7.9
EPOCHS_PER_CHECKPOINT = 20

In [ ]:
data = None
with open(path_to_module + 'light_curves_sims/separated_classes/numpy_files/dataset.pickle', 'rb') as file:
  data = pickle.load(file)
# data is list of numpy arrays:
# first array is light curve names
# second one is light curve labels
# third one is light curves
names, labels, dataset = data
labels = np.array(labels)

# normalize data using min-max range
for i, xarr in enumerate(dataset):
    mask = np.where(xarr[:, 2:3] != 0)[0]
    maxval = xarr[:, 2:3][mask].max()
    minval = xarr[:, 2:3][mask].min()
    dataset[i, :, 2:3][mask] = dataset[i, :, 2:3][mask] / (maxval-minval) 
    dataset[i, :, 3:4][mask] = dataset[i, :, 3:4][mask] / (maxval-minval)

In [ ]:
light_curve = dataset[0]
r_time, r_flux = [time for time, band, flux, error in light_curve if band == 6.5]
print(r_time)

In [ ]:
for light_curve in dataset[0:16000:800]:
  r_time = []
  r_flux = []
  g_time = []
  g_flux = []
  # print(light_curve)
  for time, band, flux, error in light_curve:
    # green band
    if band == 4.8:
      g_time.append(time)
      g_flux.append(flux)
    else:
      r_time.append(time)
      r_flux.append(flux)
  # print(r_flux)
  fig, ax1 = plt.subplots(1)
  ax1.scatter(g_time, g_flux, color = 'green')
  ax1.scatter(r_time, r_flux, color = 'red')
  fig.show()

In [ ]:
class Sampling(layers.Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = K.shape(z_mean)[0]
    dim = K.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

class Encoder(keras.Model):
  def __init__(self, shapes, mask_val=0, name = 'encoder', **kwargs):
    super(Encoder, self).__init__(name=name, **kwargs)
    self.mask_val = mask_val
    self.shapes = shapes

    # DEFINE ENCODER LAYERS
    self.enc_input = Input(shape=shapes['enc_input'])
    
    # get mask for future layer
    self.mask = Masking(mask_value=mask_val)

    # first recurrent layer
    self.gru1 = GRU(shapes['gru1'], activation='tanh', 
                    recurrent_activation='hard_sigmoid', return_sequences=True, name='gru1')
    # second recurrent layer  
    self.encoded = GRU(shapes['gru2'], activation='tanh',
                    recurrent_activation='hard_sigmoid', return_sequences=True, name='gru2')
    
    # z mean output
    self.z_mean = GRU(shapes['gru3'], return_sequences=False, activation='linear', name='gru3')

    # z variance output
    self.z_log_var = GRU(shapes['gru4'], return_sequences=False, activation='linear', name='gru4')

    # sample output
    self.z = Sampling()

  def get_config(self):
      # config = super(Encoder, self).get_config()
      config = {"shapes": self.shapes, 'mask_val':0, 'name':'encoder'}
      return config

  # define forward pass
  def call(self, inputs):
    
    mask_compute = self.mask(inputs)
    gru1 = self.gru1(mask_compute)
    mask_output = self.mask.compute_mask(gru1)

    encoded = self.encoded(gru1)
    z_mean = self.z_mean(encoded)
    z_log_var = self.z_log_var(encoded)
    z = self.z([z_mean, z_log_var])

    return z_mean, z_log_var, z

class Decoder(keras.Model):
  def __init__(self, shapes, mask_val=0, name = 'decoder', **kwargs):
    super(Decoder, self).__init__(name=name, **kwargs)
    self.mask_val = mask_val
    self.shapes = shapes

    # define layers
    self.decoder_input = Input(shape=shapes['dec_input'])

    self.repeater = RepeatVector(shapes['repeater'], name='rep')

    # time and filter ID vals
    self.input_two = Input(shape=shapes['input_two'])

    # first recurrent layer
    self.gru5 = GRU(shapes['gru5'], activation='tanh',
                    recurrent_activation='hard_sigmoid', return_sequences=True, name='gru5')
    
    # second recurrent layer
    self.gru6 = GRU(shapes['gru6'], activation='tanh',
                    recurrent_activation='hard_sigmoid', return_sequences=True, name='gru6')

    # decoder output
    self.dec_output = TimeDistributed(
        Dense(1, activation='tanh', input_shape=shapes['dec_output']), name='td')

  def get_config(self):
      config = {"shapes": self.shapes, 'mask_val':0, 'name':'decoder'}
      return config

  # define forward pass
  def call(self, inputs):
    z, train_input_two = inputs
    repeater = self.repeater(z)
    concat = concatenate((repeater, train_input_two), axis =-1)
    gru5 = self.gru5(concat)
    gru6 = self.gru6(gru5)
    dec_output = self.dec_output(gru6)

    return dec_output

class VAE(keras.Model):
  def __init__(self, prepared_data, dims=(16000,80,4), name='vae', **kwargs):
    super(VAE, self).__init__(name=name, **kwargs)
    self.epochs = 20
    self.batch_size = 64
    self.optimizer = 'adam'

    # dimension of the latent vector
    self.latent_dim = 30

    # input to first encoder and second decoder layer
    self.gru_one = 175

    # input to first decoder and second encoder layer
    self.gru_two = 150

    # load prepared dad (acts a input)
    self.prepared_data = np.array(prepared_data)
    
    # number of input features
    self.num_feats = dims[2]
    
    # number of timesteps
    self.num_timesteps = dims[1]

    # dimension of the input space for encoder
    self.enc_input_shape = (self.num_timesteps, self.num_feats)

    # number of light curves
    self.num_lcs = dims[0]

    # layer dimensions for encoder and decoder, respectively
    self.enc_dims = {
      'enc_input': self.enc_input_shape,
      'gru1': self.gru_one,
      'gru2': self.gru_two,
      'gru3': self.latent_dim,
      'gru4': self.latent_dim
    }
    self.dec_dims = {
      'dec_input': self.latent_dim,
      'repeater': self.num_timesteps,
      'input_two': (self.num_timesteps, 2),
      'gru5': self.gru_two,
      'gru6': self.gru_one,
      'dec_output': (None, 1)
    }


    # indxs for test and train
    self.train_indx = set()
    self.test_indx  = set()

    self.mask_value = 0.0
    
    self.encoder = Encoder(self.enc_dims)
    self.decoder = Decoder(self.dec_dims)

  def get_config(self):
      config = {"prepared_data": np.array(self.prepared_data), 'name':'vae'}
      return config

  # define forward pass
  def call(self, inputs):
    x_train, train_input_two = inputs
    z_mean, z_log_var, z = self.encoder(x_train)
    reconstructed = self.decoder([z, train_input_two])

    # Add KL divergence regularization loss.
    kl_loss = -0.5 * tf.reduce_mean(
        z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
    )
    self.add_loss(kl_loss)
    return reconstructed

  def reconstruction_loss(self, yTrue, yPred):
    return K.log(K.mean(K.square(yTrue - yPred)))

  def split_prep_data(self):
      """
      Splits data into 3/4 training, 1/4 testing
      """

      print("Splitting data into train and test...")

      # prepared out (only flux)
      prep_out = self.prepared_data[:, :, 2].reshape(
          self.num_lcs, self.num_timesteps, 1)
      # prep_out = self.prepared_data[:, :, 2]
      # print('prep_out shape', prep_out.shape)
      prep_inp = self.prepared_data

      x_train = []
      y_train = []
      x_test = []
      y_test = []

      # calc the # of light curves for train vs test
      num_lcs = len(prep_inp)
      train_perc = round(1.0 * num_lcs)
      test_perc = round(num_lcs*0.2)

      # save random indices for training
      while len(self.train_indx) != train_perc:
          indx = random.randint(0, num_lcs-1)
          self.train_indx.add(indx)

      # save random indices for testint -> no duplicates from training
      while len(self.test_indx) <= test_perc:
          indx = random.randint(0, num_lcs-1)
          # if indx not in self.train_indx:
          self.test_indx.add(indx)

      # extract training data
      for ind in self.train_indx:
          x_train.append(prep_inp[ind])
          y_train.append(prep_out[ind])

      # extract testing data
      for ind in self.test_indx:
          x_test.append(prep_inp[ind])
          y_test.append(prep_out[ind])

      # change to numpy arrays
      x_train = np.array(x_train).astype(np.float64)
      x_test = np.array(x_test).astype(np.float64)
      y_train = np.array(y_train).astype(np.float64)
      y_test = np.array(y_test).astype(np.float64)

      print('shape of prep_inp and x_train:', prep_inp.shape, x_train.shape)
      print('shape of prep_out and y_train:', prep_out.shape, y_train.shape)

      return x_train, x_test, y_train, y_test

  def train_model(self, x_train, x_test, y_train, y_test):
      """
      Trains the NN on training data

      Returns the trained model.
      """
      # fit model
      train_inp_two = x_train[:, :, :2]
      assert (train_inp_two.shape == (
          x_train.shape[0], x_train.shape[1], 2))

      test_inp_two = x_test[:, :, :2]
      assert (test_inp_two.shape == (
          x_test.shape[0], x_test.shape[1], 2))

      print('fitting model...')
      history = self.fit([x_train, train_inp_two], y_train, epochs=self.epochs, batch_size=self.batch_size,
                          validation_data=([x_test, test_inp_two], y_test), verbose=1, shuffle=False)

      plt.plot(history.history['loss'])
      plt.plot(history.history['val_loss'])
      plt.title('model loss')
      plt.ylabel('loss')
      plt.xlabel('epoch')
      plt.legend(['train', 'val'], loc='upper left')
      plt.show()

  def test_model(self, x_test, y_test, amount = None):
      """
      Uses test data to and NN to predict light curve decodings.

      Plots reconstructed light curved from the model prediction vs the orignal curve.
      """
      if amount:
        indices = random.sample(range(len(x_test)), k=amount)
        x_test = np.array([x_test[i] for i in indices])
        y_test = np.array([y_test[i] for i in indices])
      

      test_inp_two = x_test[:, :, :2]

      print('test_inp_one shape: ', x_test.shape)
      print('test_inp_two shape: ', test_inp_two.shape)

      self.summary()

      print('predicting...')
      for i in tqdm(range(len(x_test))):

          # predicted flux
          predicted = self.predict([x_test[i].reshape(-1, self.num_timesteps, 4),
                                    test_inp_two[i].reshape(-1, self.num_timesteps, 2)])[0]

          # if first prediction, print the prediction
          if i == 0:
              print('shape of predicted data: ', predicted.shape)

          self.plot_band_pred(y_test[i], predicted, i, test_inp_two[i])

      print("done predicting")

  def plot_band_pred(self, raw, pred, num, time_filters):
      raw_g_flux    = []
      raw_r_flux    = []
      raw_tess_flux = []

      pred_g_flux    = []
      pred_r_flux    = []
      pred_tess_flux = []

      g_time    = []
      r_time    = []
      tess_time = []
      # print(time_filters)
      for i in range(len(time_filters)):
        time, filter_ID = time_filters[i]
        raw_flux  = raw[i, 0]
        pred_flux = pred[i, 0]
        if filter_ID == 4.8:
          raw_g_flux.append(raw_flux)
          pred_g_flux.append(pred_flux)
          g_time.append(time)
        elif filter_ID == 6.5:
          raw_r_flux.append(raw_flux)
          pred_r_flux.append(pred_flux)
          r_time.append(time)
        elif filter_ID == 7.9:
          raw_tess_flux.append(raw_flux)
          pred_tess_flux.append(pred_flux)
          tess_time.append(time)

      # plot
      # make 1 x 2 figure
      fig, (ax1, ax2) = plt.subplots(2, sharey = True)
      fig.suptitle('True vs Decoded Light Curves: ' )#+ str(light_curve_names[num]))

      # pred_time = range(len(pred_flux))
      # raw_time = range(len(raw_flux))

      # plot raw data
      ax1.scatter(g_time, raw_g_flux, label='g-band', color = 'green')
      ax1.scatter(r_time, raw_r_flux, label='r-band', color = 'red')
      ax1.scatter(tess_time, raw_tess_flux, label='tess-band')
      ax1.set_ylabel('actual')

      # plot predicted data
      ax2.set_ylabel('predicted')
      ax2.scatter(g_time, pred_g_flux, label='g-band', color='green')
      ax2.scatter(r_time, pred_r_flux, label='r-band', color='red')
      ax2.scatter(tess_time, pred_tess_flux, label='tess-band')
      # save image
      fig.show()
      fig.savefig(path_to_module + "light_curve_plots/" + str(num) + ".png")
      files.download(path_to_module + "light_curve_plots/" + str(num) + ".png")
      


  def t_SNE_plot(self, labels):
      """
      Constructs 2D plots of light curves in latent space.
      """
      print('using t-SNE...')

      # extract all training label indexes
      indxs = [i for i in range(len(labels))]
      label_set = ['II', 'Ibc', 'Kilonova', 'SLSN-I', 'SNIa-91bg', 'SNIa-norm', 'SNIa-x', 'TDE']
      label_colors_map = {label:i for i, label in enumerate(label_set)}
      colors = [label_colors_map[label] for label in labels]
      # labeled_data = np.array([self.prepared_data[i] for i in indxs])

      _, _, z = tqdm(self.encoder.predict(self.prepared_data))

      t_sne = TSNE(n_components=2, learning_rate='auto',
                    init='random').fit_transform(z)
      print('t-sne shape: ', t_sne.shape)

      plt.figure(figsize=(12, 10))
      plt.scatter(t_sne[:, 0], t_sne[:, 1], c=colors)
      plt.colorbar()
      plt.title("t-SNE with only labeled data")
      plt.show()

      # # include unlabeled data
      # labels = np.array([c.loc[0, 'Class'] for c in light_curves])
      # data = self.prepared_data

      # _, _, z = tqdm(encoder.predict(data))

      # t_sne = TSNE(n_components=2, learning_rate='auto',
      #               init='random').fit_transform(z)
      # print('t-sne shape: ', t_sne.shape)

      # plt.figure(figsize=(12, 10))
      # plt.scatter(t_sne[:, 0], t_sne[:, 1], c=labels)
      # plt.colorbar()
      # plt.title("t-SNE with labeled and unlabeled data")
      # #plt.savefig(self.filepath+'plots/unlabeled-t-sne-latent-space.png', facecolor='white')
      # plt.show()

  

In [ ]:
vae = VAE(dataset, dataset.shape)
vae.encoder.summary()
vae.decoder.summary()

In [ ]:
# print(vae.prepared_data.shape)
x_train, x_test, y_train, y_test = vae.split_prep_data()
check_pt_path = path_to_module + "chck_pts_sim/separated_classes/"

# training loop:
# after training model for 20 epochs, save it. Do this 25 iterations to train for 500 epochs
optimizer = tf.keras.optimizers.Adam(clipvalue = 1.0)
vae.compile(optimizer=optimizer, loss = vae.reconstruction_loss)
vae.epochs = 10
for check_pt_numb in range(25):
  vae.train_model(x_train, x_test, y_train, y_test)
  vae.save(check_pt_path + 'ckpt_' + str(check_pt_numb))


In [ ]:
# test the trained model
x_train, x_test, y_train, y_test = vae.split_prep_data()
vae.test_model(x_test,y_test, 40)

In [ ]:
rvae = VAE(dataset, dataset.shape)
vae = keras.models.load_model(path_to_module + "chck_pts_sim/separated_classes/ckpt_11", custom_objects={
    'VAE':VAE, 'Encoder':Encoder, 
    'Decoder':Decoder, 'Sampling':Sampling,
    'reconstruction_loss':rvae.reconstruction_loss})

In [ ]:
vae.summary()
vae.encoder.summary()
# vae.encoder.input_mask.summary()

In [ ]:
# Encoder masks: used to see the masking of the layers within the model
print("ENCODER MASKS")
print(vae.encoder.layers)
vae.encoder.build(dataset.shape)
for i, l in enumerate(vae.encoder.layers):
    print(f'layer {i}: {l}')
    print(f'has input mask: {l.input_mask}')
    print(f'has output mask: {l.output_mask}')

# Decoder masks: used to see the masking of the layers within the model
print("DECODER MASKS")
for i, l in enumerate(vae.decoder.layers):
    print(f'layer {i}: {l}')
    # print(l.input_mask)
    print(f'has input mask: {l.input_mask}')
    print(f'has output mask: {l.output_mask}')

# VAE masks: used to see the masking of the layers within the model
print("VAE MASKS")
for i, l in enumerate(vae.layers):
    print(f'layer {i}: {l}')
    print(f'has input mask: {l.input_mask}')
    print(f'has output mask: {l.output_mask}')

In [ ]:
# vae.t_SNE_plot = rvae.t_SNE_plot 
rvae = VAE(dataset, dataset.shape)
vae.t_SNE_plot = rvae.t_SNE_plot
vae.t_SNE_plot(labels)

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
import pandas as pd
import numpy as np
import random
import joblib
from collections import Counter
import tensorflow as tf
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
class RandomForest:

    def __init__(self, labels, prepared_data, encoder):

        # initialize rvae
        self.rvae = VAE(prepared_data, prepared_data.shape)

        # encoded dimension from NN
        self.encoded_dim = self.rvae.latent_dim

        # get training encoder
        self.encoder = encoder

        # augmented data frame
        # self.light_curves = light_curves
        self.labels = labels

        # prepared data
        self.prepared_data = prepared_data

    def create_test_train(self):
        """
        Splits data into 85% training, 15% testing, and unlabeled
        """

        print("Splitting data for RF...")

        # extract all class outputs and inputs
        prep_out = self.labels# np.array([c.loc[0, 'Class'] for c in self.light_curves])
        prep_inp = self.prepared_data

        # extract all training label indexes
        train_indx = [i for i in range(len(prep_out)) if prep_out[i] != 3]
        unclassified_indx = [i for i in range(
            len(prep_out)) if prep_out[i] == 3]
        num_indxs = len(train_indx)

        x_train = []
        y_train = []
        x_test = []
        y_test = []
        x_unclassified = []

        # extract training data
        while len(x_train) < int(num_indxs*0.85):
            ran = random.randint(0, len(train_indx)-1)
            ind = train_indx[ran]
            train_indx.remove(ind)
            x_train.append(prep_inp[ind])
            y_train.append(prep_out[ind])

        # append the rest of the data to testing
        for ind in train_indx:
            x_test.append(prep_inp[ind])
            y_test.append(prep_out[ind])

        # extract unclassified data
        for ind in unclassified_indx:
            x_unclassified.append(prep_inp[ind])

        # change to numpy arrays
        x_train = np.array(x_train)
        x_test = np.array(x_test)
        y_train = np.array(y_train)
        y_test = np.array(y_test)
        x_unclassified = np.array(x_unclassified)

        # change y arrays to 1-d arrays
        y_train.shape = (y_train.shape[0],)
        y_test.shape = (y_test.shape[0],)

        print('shape of x_train and x_test:', x_train.shape, x_test.shape)
        print('shape of y_train and y_test:', y_train.shape, y_test.shape)
        print('shape of x_unclassified:', x_unclassified.shape)

        return x_train, x_test, y_train, y_test, x_unclassified

    def make_encodings(self, x_train, x_test, x_unclassified):
        """
        Uses trained encoder to produce 1D encodings of light curves to be used for RF training
        """
        print('making encodings...')

        # encode training light curves
        x_train_enc = self.encoder.predict(
            x_train, workers=32, use_multiprocessing=True, batch_size=128, verbose=1)[2]
        x_test_enc = self.encoder.predict(
            x_test, workers=32, use_multiprocessing=True, batch_size=128, verbose=1)[2]
        if len(x_unclassified) > 0:
          x_unclassified_enc = self.encoder.predict(
              x_unclassified, workers=32, use_multiprocessing=True, batch_size=64, verbose=1)[2]
        else:
          x_unclassified_enc = [[]]
        # x_train_enc = self.encoder.predict_on_batch(x_train)
        # x_test_enc  = self.encoder.predict_on_batch(x_test)
        # x_unclassified_enc = self.encoder.predict_on_batch(x_unclassified)
        
        # numpy arrays
        x_test_enc = np.array(x_test_enc)
        x_train_enc = np.array(x_train_enc)
        x_unclassified_enc = np.array(x_unclassified_enc)

        print('shape of encodings: ', x_train_enc.shape,
              x_test_enc.shape, x_unclassified_enc.shape)

        return x_train_enc, x_test_enc, x_unclassified_enc

    def build_classier(self, x_train, x_test, x_unclassified, y_train, y_test):
        """
        Trains a RF classifier and tests its prediction accuracy
        """
        print('building classifier...')
        rf = RandomForestClassifier(n_estimators=2000, class_weight = 'balanced')#, min_samples_split=40 )

        # reshape
        x_train = x_train.reshape(-1, self.encoded_dim)
        x_test = x_test.reshape(-1, self.encoded_dim)
        x_unclassified = x_unclassified.reshape(-1, self.encoded_dim)


        print('shape of encodings: ', x_train.shape,
              x_test.shape, x_unclassified.shape)

        # fit to data
        rf.fit(x_train, y_train)
        
        # performing predictions on the test dataset
        number_to_class = {0:'SNIa', 1:'SNIbc', 2:'SNIi', 3:'Unclassified', 4:'Other'}
        y_pred = rf.predict(x_test)

        y_train_counter = {key:value for (key, value) in Counter(y_train).items()}
        y_test_counter = {key:value for (key, value) in Counter(y_test).items()}
        y_pred_counter = {key:value for (key, value) in Counter(y_pred).items()}
        print('y_train counts: ', y_train_counter)
        print('y_test counts: ', y_test_counter)
        print('y_pred counts: ', y_pred_counter)

        # check accuracy
        print("ACCURACY OF THE MODEl: ", 100 *
              round(rf.score(x_test, y_test), 2), '%')

       # Create confusion matrix
        conf_mat = pd.crosstab(y_test, y_pred, rownames=[
                               'Actual Species'], colnames=['Predicted Species'])

        print('Confusion Matrix:')
        print(conf_mat.to_string())

        fig, ax = plt.subplots(figsize=(12, 12))
        plot_confusion_matrix(rf, x_test, y_test, display_labels = ['Ibc', 'SNIa-x', 'Kilonova', 'II', 'SNIa-norm', 'SNIa-91bg', 'SLSN-I', 'TDE'], ax=ax)
        plt.show()

        # print('Unlabeled Classifications: ')
        # unlabeled = rf.predict(x_unclassified)
        # print(unlabeled)
        # print(Counter(unlabeled))

        return rf

    def classify(self, rf, index, filename = None):
        """
        Classifies a specific light curve.
        """
        print('classifying specific light curve data...')

        # load file data if file is passed
        if filename:
            raw_df = original_curves
            names = raw_df.loc[:, 'Filename']

            for i in range(len(names)):
                if names[i] == filename:
                    indx = i

            data = self.prepared_data[indx]
            correct = raw_df.loc[indx]['Class']

        # reshape
        # data = data.reshape(1, self.rvae.num_timesteps, self.rvae.num_feats)

        # encode data
        data = self.encoder.predict(data)[2]

        # make class num -> classification dict
        classes = {0: 'SNIa', 1: 'SNIbc', 2: 'SNII',
                   3: 'Other', 4: 'Unclassified'}

        # make prediction from data
        pred = rf.predict(data)

        # print confidence
        probs = np.array(rf.predict_proba(data)[0])
        print('Number of different possible predictions: ', len(probs))
        highest_prob_ind = np.argmax(probs)
        highest_prob = max(probs)
        print('Prediction is ' + classes[highest_prob_ind]+' (',
              pred[0], ') with '+str(int(highest_prob*100))+'% confidence')
        print('Correct classification should be: ',
              classes[correct], ' (', correct, ')')

# %%

In [ ]:
# print(light_curves)
# rvae = VAE(prepared_data, prepared_data.shape)
# vae = keras.models.load_model(path_to_module + "chck_pts_sim/ckpt_100_unit_data", custom_objects={'VAE':VAE, 'Encoder':Encoder, 
#                                                                                                                               'Decoder':Decoder, 'Sampling':Sampling,
#                                                                                                                               'reconstruction_loss':rvae.reconstruction_loss})
rf= RandomForest(labels,dataset, vae.encoder)


# split data set for supervised training
x_train, x_test, y_train, y_test, x_unclassified= rf.create_test_train()
# print(x_train)
# encode input data
x_train_enc,x_test_enc,x_unclassified_enc=rf.make_encodings(x_train, x_test, x_unclassified)

# build and train the classifier
classifier = rf.build_classier(x_train_enc,x_test_enc,x_unclassified_enc,y_train,y_test)



In [ ]:
indexes=[0, 4000]
for i in indexes:
  correct = labels[i]
  # shape should be (1, 300, 4)
  light_curve = np.array([dataset[i]])
  encoded_lc = rf.encoder.predict(light_curve)[2]
  # make prediction from data
  pred = classifier.predict(encoded_lc)
  # make class num -> classification dict
  classes = {0: 'SNIa', 1: 'SNIbc', 2: 'SNII',
              3: 'Other', 4: 'Unclassified'}

  # print confidence
  probs = np.array(classifier.predict_proba(encoded_lc)[0])
  print('Number of different possible predictions: ', len(probs))
  highest_prob_ind = np.argmax(probs)
  highest_prob = probs[highest_prob_ind]
  print("Probabilities:", probs)
  print('Prediction is ' + classes[highest_prob_ind]+' (',
        pred[0], ') with '+str(int(highest_prob*100))+'% confidence')
  print('Correct classification should be: ',
        classes[correct], ' (', correct, ')')
  print()